In [0]:
%autosave 1

Autosaving every 1 seconds


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import typing as T

from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
import pyspark
import os
import numpy as np
from pyspark.ml.feature import PCA
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml.clustering import PowerIterationClustering
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import to_timestamp
from pyspark.ml.feature import PCA
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml.clustering import PowerIterationClustering
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.clustering import LDA
import numpy as np
import matplotlib.pyplot as plt


In [0]:
your_unique_name="345233563_pB" # Use this string to look for your app in the Spark UI
if your_unique_name=="noam_hw2":
    raise Exception("Please use your own name")
spark = SparkSession.builder.appName(your_unique_name).getOrCreate()

In [0]:
# Load the train and test data
prefix = "/mnt/lab94290/inputdata/part2/"
spark.conf.set("spark.sql.caseSensitive","true")
filelist = [prefix + f"requestLog_C_92004{i}.log.tar_0.json" for i in range(0,10)]
bids = spark.read.json(filelist)
bids_test = spark.read.json(prefix + 'unlabled_bids.json')

In [0]:
#bids.count()
#bids_test.count()
bids_test.printSchema()

root
 |-- app: struct (nullable = true)
 |    |-- bundle: string (nullable = true)
 |    |-- cat: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- content: struct (nullable = true)
 |    |    |-- cat: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- context: long (nullable = true)
 |    |    |-- language: string (nullable = true)
 |    |    |-- livestream: long (nullable = true)
 |    |    |-- url: string (nullable = true)
 |    |    |-- userrating: string (nullable = true)
 |    |-- domain: string (nullable = true)
 |    |-- ext: struct (nullable = true)
 |    |    |-- apilevel: long (nullable = true)
 |    |    |-- app_uptime: long (nullable = true)
 |    |    |-- app_uptime_m: long (nullable = true)
 |    |    |-- badv: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- bcat: array (nullable = true)
 |    |    |    |-- element: string (containsNull = tr

# Part 1:
## Preprocess the train and test data by :

  1) Selecting relevent features
  
  2) Filling the null values accordigly
  
  3) Indexing our string column accordigly

## 1. selecting relevent features:
It is a critical step in anomaly bid detection because it helps to reduce the complexity of the model, improve its performance, and avoid overfitting. Here are the final features we choosed:

### - Global features:

"id": used to identify unique bids and is necessary to link the bid data with the corresponding outcome .

"fake": This is the label that indicates whether a bid is fraudulent or not. It is the target variable that our model will predict.

### - Categorical feature:

"location_country": provides information about the location of the bidder, which can be used to identify suspicious patterns in bidding behavior.

### -Continous features:

"imp_pos":  position of the add on the page, which can be used to identify whether the add is being displayed in a prominent location or not.
"imp.bidfloor": minimum bid price for the impression. It is an important feature as fraudulent bids often have lower bid prices.


-Features who can be useful in identifying fraudulent behavior (fraudulent behavior refers to possible intentional deception or manipulation):

"device_connectiontype": type of connection used by the device making the bid

"device_type": type of device used to make the bid

"device_geo_type": type of geographic location data used by the device making the bid



-Features who can be useful in identifying suspicious behavior (suspicious behavior refers to patterns or characteristics that deviate from the norm and may identify suspicious patterns in bidding behavior):

"imp_topframe": whether the ad is being displayed in the top frame of the page or not

"latitude" and "longitude": geographic location of the bidder

"device_dnt": indicates whether the device making the bid has enabled the "do not track" option

"location_metro":  metropolitan area of the bidder

"spots_secure": whether the add is being served securely or not

"device_js": whether the device making the bid has JavaScript enabled

In [0]:
#1) Selecting relevent features
#-----------------train data--------------------------
dataset_q1 = bids.select(bids["id"].alias('id'),
                         bids["fake"].alias('label'),
                         bids["imp.bidfloor"].alias('bidfloor'),
                  bids["location.countryCode"].alias("location_country"),
                   bids["device.connectiontype"].alias("device_connectiontype"),
                   bids["device.devicetype"].alias("device_type"),
                   bids["imp.banner.pos"].alias("imp_pos"),
                   bids["imp.banner.topframe"].alias('imp_topframe'),
                   bids["device.geo.lat"].alias("latitude"),
                   bids["device.geo.lon"].alias("longitude"),
                   bids["device.dnt"].alias("device_dnt"),
                   bids["location.metro"].alias("location_metro"),
                   bids["spots.secure"].alias("spots_secure"),
                         bids["device.geo.type"].alias("device_geo_type"),
                         bids["device.js"].alias("device_js")
                   )
                  #Then clean the train data
dataset_q1 = dataset_q1.select("id","label",
                               F.col('bidfloor')[0].alias('bidfloor'),
                         "location_country",
                         
                         "device_connectiontype",
                         "device_type",
                         F.col("imp_pos")[0].alias('imp_pos'),
                         F.col("imp_topframe")[0].alias('imp_topframe'),
                         "latitude",
                         "longitude",
                         "device_dnt",
                         "location_metro",
                               F.col("spots_secure")[0].alias('spots_secure'),
                               "device_geo_type",
                               "device_js"
                         )

#---------test data----------------------------
# Apply the same process to the test dataset

dataset_q1_test = bids_test.select(bids_test["id"].alias('id'),
                         bids_test["imp.bidfloor"].alias('bidfloor'),
                  bids_test["location.countryCode"].alias("location_country"),
                   bids_test["device.connectiontype"].alias("device_connectiontype"),
                   bids_test["device.devicetype"].alias("device_type"),
                   bids_test["imp.banner.pos"].alias("imp_pos"),
                   bids_test["imp.banner.topframe"].alias('imp_topframe'),
                   bids_test["device.geo.lat"].alias("latitude"),
                   bids_test["device.geo.lon"].alias("longitude"),
                   bids_test["device.dnt"].alias("device_dnt"),
                   bids_test["location.metro"].alias("location_metro"),
                   bids_test["spots.secure"].alias("spots_secure"),

                         bids_test["device.geo.type"].alias("device_geo_type"),
                         bids_test["device.js"].alias("device_js")
                   )
                  
dataset_q1_test = dataset_q1_test.select("id",
                               F.col('bidfloor')[0].alias('bidfloor'),
                         "location_country",
                         
                         "device_connectiontype",
                         "device_type",
                        
                         F.col("imp_pos")[0].alias('imp_pos'),
                         F.col("imp_topframe")[0].alias('imp_topframe'),
  
                         "latitude",
                         "longitude",
                         "device_dnt",
                         "location_metro",
                               F.col("spots_secure")[0].alias('spots_secure'),
                              
                               "device_geo_type",
                               "device_js"
                         )


In [0]:
dataset_q1.printSchema()
dataset_q1_test.printSchema()

root
 |-- id: string (nullable = true)
 |-- label: boolean (nullable = true)
 |-- bidfloor: double (nullable = true)
 |-- location_country: string (nullable = true)
 |-- device_connectiontype: long (nullable = true)
 |-- device_type: long (nullable = true)
 |-- imp_pos: long (nullable = true)
 |-- imp_topframe: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- device_dnt: long (nullable = true)
 |-- location_metro: long (nullable = true)
 |-- spots_secure: long (nullable = true)
 |-- device_geo_type: long (nullable = true)
 |-- device_js: long (nullable = true)

root
 |-- id: string (nullable = true)
 |-- bidfloor: double (nullable = true)
 |-- location_country: string (nullable = true)
 |-- device_connectiontype: long (nullable = true)
 |-- device_type: long (nullable = true)
 |-- imp_pos: long (nullable = true)
 |-- imp_topframe: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable =

## 2.Filling the null values accordigly
For the train and the test data we accordigly filled the null values as follow for each one separatly:
- For Float data we filled the null data with their mean
- For the cathegorical data we remplaced the null value by the vale <UNK> so that in the string indexer process the null values of the location country would be represented 
- For binary and cathegorical numeric data like "devuce_type" or "device_js" we filles the na values by thr most occurent value in the  data.

In [0]:
#2)filling the null values accordigly
#----------train----------
# Analyse  of the numericals features


from pyspark.sql.functions import mean

# Return the mean of the selected column 
def test_data(column):
    test = dataset_q1.select(dataset_q1[column].alias(column))
    test.select(mean (column)).show()
    
    
# Return a sorted descending summarize of the selected column 
def test_data2(column):
    test = dataset_q1.select(dataset_q1[column].alias(column))
    test.groupBy(column).count().orderBy('count', ascending=False).show()
columns = ['bidfloor',"device_connectiontype","device_type",

                         'imp_pos',
                         'imp_topframe',
                         "latitude",
                         "longitude",
                         "device_dnt",
                         "location_metro",
                               'spots_secure',
                               "device_geo_type",
                               "device_js"]#,"location_country"
for col in columns:
    test_data(col)
    test_data2(col)
    print("fast")

+------------------+
|     avg(bidfloor)|
+------------------+
|2.2323864933140722|
+------------------+

+-------------------+------+
|           bidfloor| count|
+-------------------+------+
|               null|180331|
|               0.01|155061|
|0.12000000000000001|124243|
|            0.08519|102465|
|               0.02| 91775|
|             1.0E-4| 54450|
|              0.226| 34569|
|               0.16| 33769|
|       0.7178823528| 31623|
|0.13440000000000002| 31141|
|           0.283562| 26520|
|           0.114286| 25769|
|              0.675| 22332|
|           0.224286| 22240|
|            4.52128| 20032|
|            2.26564| 19952|
|              1.215| 19281|
|              0.125| 18354|
|               0.54| 18202|
|              0.252| 17094|
+-------------------+------+
only showing top 20 rows

fast
+--------------------------+
|avg(device_connectiontype)|
+--------------------------+
|         2.943611382868867|
+--------------------------+

+--------------------

In [0]:
#fill by the mean if float and by the most wiewed in int and booo
dataset_q1 = dataset_q1.fillna(2.2323864933139697, ['bidfloor'] )
dataset_q1 = dataset_q1.fillna("UNK", ['location_country'] )

dataset_q1 = dataset_q1.fillna( 2, ['device_connectiontype'] )
dataset_q1 = dataset_q1.fillna(4, ['device_type'] )

dataset_q1 = dataset_q1.fillna(0, ['imp_pos'] )
dataset_q1 = dataset_q1.fillna(0, ['imp_topframe'] ) 

dataset_q1 = dataset_q1.fillna(23.304048203284903, ['latitude'] )
dataset_q1 = dataset_q1.fillna(32.33560026254626, ['longitude'] )

dataset_q1 = dataset_q1.fillna(0, ['device_dnt'] )
dataset_q1 = dataset_q1.fillna(1, ['spots_secure'] )
dataset_q1 = dataset_q1.fillna(-1, ['location_metro'] ) #not needed but just in case
dataset_q1 = dataset_q1.fillna(2, ["device_geo_type"] )
dataset_q1 = dataset_q1.fillna(0, ["device_js"] )
dataset_q1 = dataset_q1.na.drop()#not needed but just in case




In [0]:
#----------test----------------

from pyspark.sql.functions import mean
def test_data(column):
    test = dataset_q1_test.select(dataset_q1_test[column].alias(column))
    test.select(mean (column)).show()
def test_data2(column):
    test = dataset_q1_test.select(dataset_q1_test[column].alias(column))

    test.groupBy(column).count().orderBy('count', ascending=False).show()
columns = ['bidfloor',"device_connectiontype","device_type",

                         'imp_pos',
                         'imp_topframe',
                         
                         "latitude",
                         "longitude",
                         "device_dnt",
                         "location_metro",
                               'spots_secure',

                               "device_geo_type",
                               "device_js"]
for col in columns:
    test_data(col)
    test_data2(col)
    print("fast")





+------------------+
|     avg(bidfloor)|
+------------------+
|2.1353123963798653|
+------------------+

+-------------------+-----+
|           bidfloor|count|
+-------------------+-----+
|0.12000000000000001| 3192|
|               null| 3106|
|               0.01| 2932|
|            0.08519| 2451|
|               0.02| 1513|
|       0.7912207788| 1038|
|               0.16|  865|
|             1.0E-4|  855|
|              0.226|  802|
|           0.114286|  732|
|       0.7178823528|  675|
|0.13440000000000002|  666|
|           0.283562|  624|
|           0.224286|  617|
|              0.675|  581|
|              0.125|  501|
|            4.52128|  438|
|0.16038000000000002|  437|
|              1.215|  435|
|           2.653425|  430|
+-------------------+-----+
only showing top 20 rows

fast
+--------------------------+
|avg(device_connectiontype)|
+--------------------------+
|        2.9012204291746198|
+--------------------------+

+---------------------+-----+
|device_connect

In [0]:
#fill by the mean if double and by the most wiewed in int and bool
dataset_q1_test = dataset_q1_test.fillna(2.13531239637988, ['bidfloor'] )
dataset_q1_test = dataset_q1_test.fillna("UNK", ['location_country'] )

dataset_q1_test = dataset_q1_test.fillna( 2, ['device_connectiontype'] )
dataset_q1_test = dataset_q1_test.fillna(4, ['device_type'] )

dataset_q1_test = dataset_q1_test.fillna(0, ['imp_pos'] )
dataset_q1_test = dataset_q1_test.fillna(0, ['imp_topframe'] ) 

dataset_q1_test = dataset_q1_test.fillna(22.82185997087692, ['latitude'] )
dataset_q1_test = dataset_q1_test.fillna(31.84073287691015, ['longitude'] )

dataset_q1_test = dataset_q1_test.fillna(0, ['device_dnt'] )
dataset_q1_test = dataset_q1_test.fillna(1, ['spots_secure'] )
dataset_q1_test = dataset_q1_test.fillna(-1, ['location_metro'] ) 
dataset_q1_test = dataset_q1_test.fillna(2, ["device_geo_type"] )
dataset_q1_test = dataset_q1_test.fillna(0, ["device_js"] )
dataset_q1_test = dataset_q1_test.na.drop() # Not needed but just in case


## 3.indexing our string column:
 
We use se a StringIndexer transformer from PySpark's Machine Learning library to transform the categorical into a numerical variable. This transformer is fit on the training data and then applied on both the training and test data.

In [0]:
#3) Indexing our string columns accordigly

#-------train&test----------------
indexer_country = StringIndexer(inputCol='location_country', outputCol='location_country_id').fit(dataset_q1)
indexed_country = indexer_country.transform(dataset_q1)
indexed_country_test = indexer_country.transform(dataset_q1_test)
dataset_q1 = indexed_country.drop('location_country')
dataset_q1_test = indexed_country_test.drop('location_country')
#show the number of row we use in total (it should be all the rows)
dataset_q1.count()
dataset_q1_test.count()

Out[13]: 90857

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType

# Cast the label column into double for enabeling it to be used as a label in the algorithms

dataset_q1 = dataset_q1.withColumn("label",col("label").cast("double")) 

In [0]:
# Prepare the data for supervised learning model

column_selected = ['bidfloor',"location_country_id","device_connectiontype","device_type",
                         'imp_pos',
                         'imp_topframe',
                         "latitude",
                         "longitude",
                         "device_dnt",
                         "location_metro",
                               'spots_secure',
                               "device_geo_type",
                               "device_js"]

# The "features" vector will be used as input to predict the "label" column, which represents whether or not a bid is fraudulent
vector_assembler = VectorAssembler(inputCols=column_selected, outputCol='features')
df_train_final = vector_assembler.transform(dataset_q1).select('id', 'features', 'label')
df_test_final = vector_assembler.transform(dataset_q1_test).select('id', 'features')



In [0]:
df_train_final.show()
df_test_final.show()

+--------------------+--------------------+-----+
|                  id|            features|label|
+--------------------+--------------------+-----+
|376cba30-9654-a04...|[3.59307,0.0,3.0,...|  0.0|
|a5f2481b-5e0a-6e4...|[2.23238649331396...|  0.0|
|606147f0-9c5c-0d4...|[1.837485,0.0,3.0...|  0.0|
|b6dc5c73-a442-414...|[2.653425,10.0,5....|  0.0|
|83abb2c9-52f3-ff4...|[17.68,9.0,2.0,4....|  0.0|
|6f0205b4-555c-a14...|[4.219,1.0,2.0,4....|  0.0|
|4d9f9bd3-b682-3a4...|[0.76188,4.0,6.0,...|  0.0|
|3eb8aab7-b989-a24...|[0.02,9.0,2.0,4.0...|  0.0|
|94998f6d-b56c-2b4...|[0.15,8.0,6.0,4.0...|  0.0|
|971a99a1-73db-a34...|[0.083562,10.0,2....|  0.0|
|5254aef8-0c65-de4...|[0.731944,0.0,3.0...|  0.0|
|169d656a-5171-314...|[0.4113,41.0,2.0,...|  0.0|
|ee24eaf5-5d64-6d4...|[1.10315520000000...|  0.0|
|878aa7e9-ea42-e14...|[1.08476928000000...|  0.0|
|c7272a4d-a27b-ac4...|[2.23238649331396...|  0.0|
|c94fa5a7-19b1-474...|[0.05461538461538...|  0.0|
|727be4e9-dbae-2c4...|[0.114286,1.0,2.0...|  0.0|


#Part 2: train and evaluate the model:

  1) training the model: split data between train and val to train and estimate our model
  
  2) find predictions using our model(evaluate val_data and predict on the test data)
  
  3) take our preduction of the test data and save it as csv as required:

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import confusion_matrix
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier

# Split the train data into training and validation sets
split_ratio = [0.8, 0.2] # 80% for training, 20% for validation

# Seed is to ensure refesability
train_data, validation_data = df_train_final.randomSplit(split_ratio, seed=1234)


#Model 1: LogisticRegression

### 1. How does the algorithm work?

Logistic Regression is a linear classification algorithm used for binary classification problems, where the goal is to predict a binary outcome (i.e., 0 or 1) based on a set of features. 

It models the probability of the outcome using a logistic function, which maps any input to the range of [0, 1].

The algorithm works by minimizing a loss function that penalizes the model for misclassifying instances, while also regularizing the model to prevent overfitting. The learned model can then be used to predict the probability of an instance being normal or anomalous. 

### 2. Why did we choosed this model? 

Because it is a basic yet efficient method that can be trained fast and doesn't require a lot of training data. 

In addition, it can handle relationships that are both linear and nonlinear between the features and the results, and it is simple to read in order to get knowledge about the significance of various features in identifying abnormal behavior.

Finaly, in the context of anomaly detection, the Logistic Regression model can be used to learn the normal patterns of behavior in the data, and then identify instances that deviate significantly from the learned pattern. 

Also, a variety of datasets, including ones with high-dimensional feature spaces, have demonstrated it to perform well. As a result, it is a dependable option for anomaly detection and may be an excellent place to begin researching

In [0]:
# Create a logistic regression model and train the relevent data
logistic_regression = LogisticRegression(featuresCol='features', labelCol='label')
model = logistic_regression.fit(train_data)

# Make predictions on the validation data using the trained model
predictions = model.transform(validation_data)

#  Example rows of the predictions
predictions.select("id","features","label","prediction").show(5)

# Evaluate the accuracy of the predictions
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# Compute the confusion matrix and metrics
pred = predictions.select("prediction").collect()
true = predictions.select("label").collect()
confusion_mat = confusion_matrix(true, pred)

print("Confusion Matrix: ")
TP = confusion_mat[0][0]
FP = confusion_mat[0][1]
FN = confusion_mat[1][0]
TN = confusion_mat[1][1]
print("TP,FP,FN,TN: ",TP,FP,FN,TN)

precision = TP/(TP+FP)
recall = TP/(TP+FN)
metrics = 5*precision*recall /(4*precision +recall)
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("Final metric: ", metrics)


+--------------------+--------------------+-----+----------+
|                  id|            features|label|prediction|
+--------------------+--------------------+-----+----------+
|0001be52-f18e-ad4...|[0.01689552238805...|  0.0|       0.0|
|00037ae2-ef58-854...|[0.02205128205128...|  0.0|       0.0|
|0010b4bf-42a2-024...|[5.0,8.0,6.0,4.0,...|  0.0|       0.0|
|001138e3-1856-ee4...|[0.01,7.0,6.0,4.0...|  0.0|       0.0|
|001358b3-1155-f84...|[2.23238649331396...|  0.0|       0.0|
+--------------------+--------------------+-----+----------+
only showing top 5 rows

Confusion Matrix: 
TP,FP,FN,TN:  798024 33 4067 5
Accuracy:  0.9948886027060485
Precision:  0.9999586495701435
Recall:  0.9949295030115037
Final metric:  0.995931281310676


#Model 2: GBTClassifier

### 1. How does the algorithm work?

  Gradient Boosting is an ensemble machine learning algorithm that combines multiple decision trees (weak learners) to create a strong learner. The algorithm works by building each subsequent weak learner on the difference between the predicted values and the actual values of the previous weak learner.

  GBTClassifier, in particular, is a variant of Gradient Boosting that is used for binary classification problems. The algorithm works by iteratively building decision trees and adding them to an ensemble, each tree attempting to correct the mistakes of the previous ones. The final ensemble is a combination of all the individual decision trees, which are weighted based on their performance in improving the model's predictions.
  

### 2. Why did we choosed this model? 

  In the context of anomaly detection, the GBTClassifier model can be used to learn the normal patterns of behavior in the data and identify instances that deviate significantly from the learned pattern. The algorithm works by optimizing a loss function that penalizes the model for misclassifying instances, while also regularizing the model to prevent overfitting.

  In addition It is able to handle complex and non-linear relationships between the input features and the target variable. Crucial in anomaly detection, because an anomaly can be influed by multiple factors. 

  Furthermore, It is robust to noise and outliers in the data. By using multiple decision trees and assigning weights to the instances, the algorithm is able to handle noisy data and improve the accuracy of the predictions.

  In summary,the GBTClassifier is a strong and efficient machine learning technique for supervised anomaly detection.
For our anomaly detection jobs, it is an excellent option due to its capacity for handling complex relationships, robustness to noise, interpretability, scalability, and flexibility.

In [0]:
# Train a Gradient Boosted Tree Classifier model 
gbtc = GBTClassifier(labelCol="label", maxIter=30, maxBins=256)
model = gbtc.fit(train_data)

# Make predictions on the validation and test datasets
val_predictions = model.transform(validation_data)
test_predictions = model.transform(df_test_final)

# Evaluate the model's accuracy using a MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(val_predictions)

# Calculate evaluation metrics using the confusion matrix
pred = val_predictions.select("prediction").collect()
true = val_predictions.select("label").collect()
confusion_mat = confusion_matrix(true, pred)
TP = confusion_mat[0][0]
FP = confusion_mat[0][1]
FN = confusion_mat[1][0]
TN = confusion_mat[1][1]

print("TP,FP,FN,TN: ",TP,FP,FN,TN)

precision = TP / (TP + FP)
recall = TP / (TP + FN)
metrics = 5 * precision * recall / (4 * precision + recall)

# Print evaluation metrics
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("Final metric: ", metrics)


Accuracy:  0.9976537439738495
Precision:  0.9997995130673624
Recall:  0.9978464743834251
Final metric:  0.9982364714616393


The GBTClassifier model outperformed the LogisticRegression model in the anomaly detection problem.

Indeed, the relationships between the selected features and the target variable is be complex and nonlinear, which the GBTClassifier can handle better than LogisticRegression, the uses of ensemble learning by The GBTClassifier improve accuracy and reduce overfitting. 

Furthermore the GBTClassifier is more robust to outliers in the data, making it more resilient to anomalies or outliers that could disrupt the modeling process.

On top of that, our final metric is better in many way with the GBTC, indeed it has a better overall result , has also a better precision

Pressision here is important because the formula for this metric involves weighting precision more heavily than recall, with a ratio of 5:4 which is useful in scenarios where detecting false positives is more important than missing true positives.
An explication of the choice of this metric may bebecause when precision becomes more important than recall it help in detecting false positives that is critical to avoiding financial losses.

In conclusion because we got better results in both accuracy and the final metrics with theGBTClassification model we will return his results for both questions

In [0]:
# Select the IDs of anomalous bids from the test data predictions
anomaly_ids = test_predictions.filter(test_predictions.prediction == 1).select('id')

# Rename the "id" column to "anomaly"
anomaly_ids = anomaly_ids.withColumnRenamed('id', 'anomaly')

# Create a new dataframe containing only the "anomaly" column
anomaly_pred = anomaly_ids.select('anomaly')


#save it as csv as required:
anomaly_pred.show()



+--------------------+
|             anomaly|
+--------------------+
|afd33054-95e4-464...|
|3c94b0e4-f2a7-fd3...|
|1a136245-ff16-3bc...|
|67012b8f-a24e-0b1...|
|a451fab0-92d2-5da...|
|328770e2-268d-971...|
|e4abe6e6-b770-f97...|
|d83fe14a-58a3-69c...|
|89ec24b2-0afb-4d3...|
|ad8a4046-c595-a0b...|
|4202992a-bcfd-b8d...|
|f2c8e8a6-d9b6-730...|
|37b63599-0c34-732...|
|d2b72cd2-8e04-51a...|
|c0bfe94f-008e-6de...|
|ef38b626-ed83-4af...|
|820be1c9-9e86-094...|
|d3241174-1801-f0f...|
|79f446d2-46de-f7a...|
|8de7567c-ffb2-3ad...|
+--------------------+
only showing top 20 rows



In [0]:
# Question 1:
anomaly_pred.write.csv("/dbfs/mnt/lab94290/results/q1_345233563.csv",mode = "overwrite")

# Question 2:
anomaly_pred.write.csv("/dbfs/mnt/lab94290/results/q2_345233563.csv",mode = "overwrite")